In [0]:
SELECT * FROM silver.silver_imdb_movies_clean LIMIT(5);

SELECT * FROM silver.silver_imdb_reviews_clean LIMIT(5);

SELECT * FROM silver.silver_imdb_reviews_enriched LIMIT(5);



CREATE OR REPLACE TABLE gold.gold_movie_performance AS
SELECT
    r.imdb_id,
    m.title,
    m.genre,
    m.year,

    COUNT(*) AS total_reviews,
    AVG(r.review_rating) AS avg_review_rating,
    AVG(r.sentiment_score) AS avg_sentiment,

    AVG(CASE WHEN r.sentiment_label = 'positive' THEN 1 ELSE 0 END) AS positive_ratio,
    AVG(CASE WHEN r.sentiment_label = 'negative' THEN 1 ELSE 0 END) AS negative_ratio,
    AVG(CASE WHEN r.sentiment_label = 'neutral'  THEN 1 ELSE 0 END) AS neutral_ratio,

    CASE
        WHEN AVG(r.review_rating) >= 4
         AND AVG(CASE WHEN r.sentiment_label = 'positive' THEN 1 ELSE 0 END) >= 0.6
        THEN 'Hit'
        WHEN AVG(r.review_rating) >= 3
        THEN 'Average'
        ELSE 'Flop'
    END AS overall_verdict

FROM silver.silver_imdb_reviews_enriched r
JOIN silver.silver_imdb_movies_clean m
  ON r.imdb_id = m.id
GROUP BY r.imdb_id, m.title, m.genre, m.year;
SELECT * FROM gold.gold_movie_performance LIMIT(5);




CREATE OR REPLACE TABLE gold.gold_genre_insights AS
SELECT
    genre,
    COUNT(DISTINCT imdb_id) AS total_movies,
    AVG(avg_review_rating) AS avg_rating,
    AVG(positive_ratio) AS avg_positive_ratio,
    AVG(negative_ratio) AS avg_negative_ratio
FROM gold.gold_movie_performance
GROUP BY genre;

SELECT * FROM gold.gold_genre_insights LIMIT(5);




CREATE OR REPLACE TABLE gold.gold_sentiment_mismatch AS
SELECT *,
       ABS(avg_review_rating - avg_sentiment) AS sentiment_rating_gap,
       CASE
           WHEN ABS(avg_review_rating - avg_sentiment) > 1
           THEN 'Mismatch'
           ELSE 'Aligned'
       END AS alignment_flag
FROM gold.gold_movie_performance;
SELECT * FROM gold.gold_sentiment_mismatch LIMIT(5);





CREATE OR REPLACE TABLE gold.gold_polarizing_content AS
SELECT *
FROM gold.gold_movie_performance
WHERE positive_ratio >= 0.4
  AND negative_ratio >= 0.4;
SELECT * FROM gold.gold_polarizing_content LIMIT(5);